# Access Dataset
tensorflow-dataset food101

In [ ]:
## uncomment this line if not using google colab
# pip install tensorflow-datasets

In [ ]:
## import library
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import os.path
from os import path

In [ ]:
food101_builder = tfds.builder("food101")
food101_info = food101_builder.info

In [ ]:
def download_food101_dataset():
  food101_builder.download_and_prepare(
    # download_dir = 'drive/MyDrive/Capstone/Dataset/Food101',
    download_config=None)

In [ ]:
download_food101_dataset()

In [ ]:
datasets = food101_builder.as_dataset(
    split=None,
    batch_size=None,
    shuffle_files=False,
    read_config=None,
    as_supervised=True)

train_dataset, val_dataset = datasets["train"], datasets["validation"]
assert isinstance(train_dataset, tf.data.Dataset)

In [ ]:
## print dataset type
ds_train = train_dataset
print(ds_train)

In [ ]:
## print the metadata
ds_info = food101_info
print(ds_info)

# Pre-Processing

In [ ]:
def normalize_img(image, label):
    image = tf.cast(image, tf.float32)
    # Normalize the pixel values
    image = image / 255.0
    # Resize the image
    image = tf.image.resize(image, (224, 224))   # 224 is default
    label = tf.expand_dims(label,axis=-1)
    label = tf.cast(label, tf.int32)
    return image, label

In [ ]:
## normalize train dataset
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# ds_train = ds_train.cache()
# ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
## normalize validation dataset
ds_val = val_dataset
ds_val = ds_val.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(128, drop_remainder=True)
# ds_val = ds_val.cache()
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
## check datatype after normalization
print(ds_train)
print(ds_val)

# Visualization data

In [ ]:
ds, info = tfds.load(name='food101', 
                     split='train', 
                     # data_dir = 'drive/MyDrive/Capstone/Dataset',
                     download=False,
                     with_info=True)

In [ ]:
## show some examples images
fig = tfds.show_examples(ds, info)

# Train Model
Pick one manual or trf learning

## Manual



In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(101, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam' , loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(ds_train, epochs=2, validation_data=ds_val, verbose=1)

## Transfer Learning

In [ ]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
def build_model_mobile():
       # constructing the model
    model = keras.applications.mobilenet.MobileNet(weights="imagenet", 
                                                   include_top=False, 
                                                   input_shape=(224, 224, 3),
                                                   pooling='avg')
    for layer in model.layers:
        layer.trainable = False

    # Adding custom Layers
    x = model.output
    x = Dense(128, activation="relu")(x)
    x = Dense(128, activation="relu")(x)
    predictions = Dense(101, activation="softmax")(x)

    # creating the final model
    model_final = Model(inputs=model.input, outputs=predictions)
    
    return model_final

model_mobile = build_model_mobile()
model_mobile.compile(loss=sparse_categorical_crossentropy, optimizer='adam', metrics=['acc'])

In [ ]:
model_mobile.summary()

In [ ]:
history = model_mobile.fit(ds_train, epochs=1, validation_data=ds_val, verbose=1)

# Prediction

In [ ]:
## make dictionary for label
label_path = '/root/tensorflow_datasets/food101/2.0.0/label.labels.txt'
dict_label = {}
i = 0

with open(label_path) as f:
    for line in f:
      dict_label[int(i)] = line.strip('\n')
      i += 1

print(dict_label)

In [ ]:
## upload image for prediction
from google.colab import files
im = files.upload()

In [ ]:
## making prediction
from PIL import Image
import numpy as np
from keras.preprocessing import image
for fn in im.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  
  pred=model_mobile.predict([images])[0]
  print(pred) 

In [ ]:
## get the prediction based on the max value
max_value = np.max(pred)
print(max_value)
predic = np.argmax(pred)
print(predic)

print('Image = ' + dict_label[predic])

# Save Model

In [ ]:
model = model_mobile
model.input

In [ ]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)

tfmodel = converter.convert()

open('food.h5', 'wb').write(tfmodel)